# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Streaming_Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final project: Machine Learning** </center>
<div style="text-align: center;">
    <br>
    <strong>Estudiante:</strong> Juan Pablo Quintero <br>
    <strong>Fecha:</strong> 23 de Noviembre de 2025 <br>
    <strong>Profesor:</strong> Pablo Camarrilo Ramirez 

# Iniciamos Spark

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ML:Proyecto Final_Random Forest") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/25 02:29:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# ML Algoritmo a Usar

Voy usar el Ml de clasificacion, debido a que quiero ver el nivel de trafico que enfrenta un repartidor de comida rapida.
Este problema me parece interesnate por que quien no a pedido por una aplicacion de comida rapida o a pedido un servicio de comida rapida yo creo que todos, entonces para resolver la poblematica de que la comida te llegue fria , es crucial saber si el trafico va a estar **Alto**, **Medio**, o **Bajo** esto con la unnica finalidad de la satisfaccion del cliente y que este al tanto de que tanto podria tardar su pedido por el trafico. <br>
El algoritmo seleccionado : **Random Forest**, elegi random forest por que es bueno tanto manejando variables numericas como la distancia y categoricas como el clima, y la zona.

# Descripcion del Dataset

In [6]:
from PabloQuintero.spark_utils import SparkUtils
#Definimos el Dataset
Repartidor_schema = SparkUtils.generate_schema([
    ("order_id", "IntegerType"),
    ("distance_km", "FloatType"),
    ("delivery_time_min", "FloatType"),
    ("traffic_level", "StringType"),
    ("route_length_km", "FloatType"),
    ("delivery_mode", "StringType"),
    ("weather", "StringType"),
    ("order_time", "TimestampType"),
    ("restaurant_zone", "StringType"),
    ("customer_zone", "StringType")
])
Repartidor_df = spark.read \
                    .option("header", "true") \
                    .schema(Repartidor_schema) \
                    .csv("/opt/spark/work-dir/data/ml/Random_Forest/")
Repartidor_df.printSchema()
Repartidor_df.show()
#Tamaño del dataset
print(f"Total de registros: {Repartidor_df.count()}")
print(f"Columnas: {len(Repartidor_df.columns)}")
#Esta balanceado ? Usando el nivel del trafico
Repartidor_df.groupBy("traffic_level").count().show()


root
 |-- order_id: integer (nullable = true)
 |-- distance_km: float (nullable = true)
 |-- delivery_time_min: float (nullable = true)
 |-- traffic_level: string (nullable = true)
 |-- route_length_km: float (nullable = true)
 |-- delivery_mode: string (nullable = true)
 |-- weather: string (nullable = true)
 |-- order_time: timestamp (nullable = true)
 |-- restaurant_zone: string (nullable = true)
 |-- customer_zone: string (nullable = true)

+--------+-----------+-----------------+-------------+---------------+-------------+-------+-------------------+---------------+-------------+
|order_id|distance_km|delivery_time_min|traffic_level|route_length_km|delivery_mode|weather|         order_time|restaurant_zone|customer_zone|
+--------+-----------+-----------------+-------------+---------------+-------------+-------+-------------------+---------------+-------------+
|       1|       7.97|             63.8|         High|           9.75|      Bicycle|  Clear|2025-01-01 15:29:00|          

# ML Proceso de entrenamineto 

In [12]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
#Preparando los datos 
ind_weather = StringIndexer(inputCol="weather", outputCol="weather_idx")
ind_mode = StringIndexer(inputCol="delivery_mode", outputCol="delivery_mode_idx")
ind_Rzone = StringIndexer(inputCol="restaurant_zone", outputCol="restaurant_zone_idx")

indexer_label = StringIndexer(inputCol="traffic_level", outputCol="label")

assembler = VectorAssembler(
    inputCols=["distance_km", "weather_idx", "delivery_mode_idx", "restaurant_zone_idx"],
    outputCol="features"
)
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=20, maxDepth=5)
pipeline = Pipeline(stages=[ind_weather, ind_mode, ind_Rzone, indexer_label, assembler, rf])
train_data, test_data = Repartidor_df.randomSplit([0.7, 0.3], seed=42)
model = pipeline.fit(train_data)
#Guardar el Modelo
model.write().overwrite().save("food_delivery_rf_model")

25/11/25 03:17:35 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

# ML Evaluacion

In [13]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#Predicciones
predictions = model.transform(test_data)
predictions.select("traffic_level", "label", "prediction", "probability").show(5)
#Evaluar modelo
evaluator_acc = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_acc.evaluate(predictions)
print(f"Accuracy del modelo: {accuracy:.2f}")

evaluator_f1 = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="f1")

f1_score = evaluator_f1.evaluate(predictions)
print(f"F1-Score del modelo: {f1_score:.2f}")

+-------------+-----+----------+--------------------+
|traffic_level|label|prediction|         probability|
+-------------+-----+----------+--------------------+
|       Medium|  2.0|       1.0|[0.34844322344322...|
|          Low|  0.0|       0.0|[0.79827393971223...|
|       Medium|  2.0|       1.0|[0.35700874615580...|
|         High|  1.0|       2.0|[0.17083516483516...|
|          Low|  0.0|       2.0|[0.20793452380952...|
+-------------+-----+----------+--------------------+
only showing top 5 rows
Accuracy del modelo: 0.29
F1-Score del modelo: 0.29
